<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Refactoring" data-toc-modified-id="Refactoring-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Refactoring</a></span></li></ul></div>

# Refactoring

As the code may be modified later, I want to refactor from the jupyter notebook to a Python script. Thus I create a file ``main.py``:

In [2]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize


def read_csv(filename: str) -> pd.DataFrame:
    df = pd.read_csv(filename)
    return df


def clean_text(df: pd.DataFrame) -> list:
    texts = [i.lower() for i in df.text]
    remove_equations = [re.sub("\$.*?\$", "", s) for s in texts]
    remove_tags = [re.sub("<.*?>", "", s) for s in remove_equations]
    remove_percentages = [re.sub("%", "", s) for s in remove_tags]
    remove_links = [re.sub(r"http\S+", "http", s) for s in remove_percentages]
    remove_formulas = [
        re.sub("[a-z]*[0-9]* *\~([a-z]*[0-9]* *\+)* *[a-z]*[0-9]*", "", s)
        for s in remove_links
    ]
    return [re.sub("[a-z_]*=['|\"]*[0-9a-z_]*", "", s) for s in remove_formulas]


def tokenize_texts(texts: list, df: pd.DataFrame) -> pd.DataFrame:
    df["text_tokenized"] = [" ".join(word_tokenize(s)) for s in texts]
    return df


if __name__ == "__main__":
    filename = "../input/raw/stackexchange_812k.csv"
    data = read_csv(filename)
    texts = clean_text(data)
    df = tokenize_texts(texts, data)
    df.to_csv("../input/preprocessed/tokenized.csv")

In [3]:
df.head()

,post_id,parent_id,comment_id,text,category,text_tokenized
0,1,NaN,NaN,Eliciting priors from experts,title,eliciting priors from experts
1,2,NaN,NaN,What is normality?,title,what is normality ?
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,what are some valuable statistical analysis op...
3,4,NaN,NaN,Assessing the significance of differences in d...,title,assessing the significance of differences in d...
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,the two cultures : statistics vs. machine lear...


In [ ]:
# 